<a href="https://colab.research.google.com/github/ab-sa/Statistical-Machine-Learning3/blob/main/Lecture5_PCR_PLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
#import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline

Import Credit data

In [ ]:
Credit = pd.read_csv('Credit.csv')
print('Dimension of the data: ' + str(Credit.shape))
Credit.head()

PCR

In [ ]:
y = Credit['Balance']
X = Credit[['Limit', 'Rating', 'Cards', 'Age', 'Education']]

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

std_scale = StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

pca = PCA()
PCs_train = pca.fit_transform(X_train_std)
PCs_test = pca.transform(X_test_std)

LModel = linear_model.LinearRegression()
MSPEs = []

# Calculate MSPE with only the intercept
#LModel.fit(np.ones((len(PCs_train),1)), y_train)
#MSPEs.append(mean_squared_error(y_test, LModel.predict(np.ones((len(PCs_test),1)))))

# Calculate MSPE: adding one component at a time
for i in np.arange(1, 6):
  LModel.fit(PCs_train[:,:i], y_train)
  MSPEs.append(mean_squared_error(y_test, LModel.predict(PCs_test[:,:i])))

print(MSPEs)

# Plot MSPEs results    
plt.plot(MSPEs)
plt.xlabel('Number of Principal Components')
plt.ylabel('MSPE')

In [ ]:
LModel.fit(PCs_train[:,:4], y_train)
print('MSPE: ', mean_squared_error(y_test, LModel.predict(PCs_test[:,:4])))
print('Coef estimatees: ', LModel.coef_)

Create dummy variavles and initialize 10-fold CV

In [ ]:
# from numpy.core.fromnumeric import shape
Credit_dumms = pd.get_dummies(Credit)
Credit_dumms['CV'] = np.random.randint(low=0, high=10, size=(Credit_dumms.shape[0],))
print(Credit_dumms['CV'].value_counts())
print(Credit_dumms.isnull().sum())
Credit_dumms

Model comparison:

In [ ]:
Ridge_MSPE = []
LASSO_MSPE = []
LASSO_NFeat = []
PCR_MSPE = []
PCR_NCopm = []
PLS_MSPE = []
PLS_NCopm = []

for i in range(10):
  Credit_dumms_train = Credit_dumms.loc[Credit_dumms['CV'] != i]
  Credit_dumms_test = Credit_dumms.loc[Credit_dumms['CV'] == i]

  y_train = Credit_dumms_train['Balance']
  X_train = Credit_dumms_train.drop(['ID', 'Balance', 'CV'], axis=1)
  y_test = Credit_dumms_test['Balance']
  X_test = Credit_dumms_test.drop(['ID', 'Balance', 'CV'], axis=1)

  std_scale = StandardScaler().fit(X_train)
  X_train_std = std_scale.transform(X_train)
  X_test_std  = std_scale.transform(X_test)

  CV10 = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)

  # Ridge
  LMRidgeCV = RidgeCV(alphas=np.arange(0.5, 2, 0.01), cv=CV10, scoring='neg_mean_squared_error')
  LMRidgeCV.fit(X_train_std, y_train)
  LMRidge = Ridge(alpha = LMRidgeCV.alpha_)
  LMRidge.fit(X_train_std, y_train)
  Ridge_MSPE.append(mean_squared_error(y_test, LMRidge.predict(X_test_std)))

  # LASSO
  LMLassoCV = LassoCV(alphas = np.arange(0.5, 2, 0.01), cv = CV10, max_iter = 1000)
  LMLassoCV.fit(X_train_std, y_train)
  LMLasso = Lasso(max_iter = 10000)
  LMLasso.set_params(alpha=LMLassoCV.alpha_)
  LMLasso.fit(X_train_std, y_train)
  LASSO_MSPE.append(mean_squared_error(y_test, LMLasso.predict(X_test_std)))
  LASSO_NFeat.append(len(LMLasso.coef_) - sum(LMLasso.coef_ == 0))

  # PCR
  pca = PCA()
  LModel = linear_model.LinearRegression()
  for pcr_train_index, pcr_test_index in CV10.split(X_train_std):
    pcr_X_train, pcr_X_test = X_train_std[pcr_train_index], X_train_std[pcr_test_index]
    pcr_y_train, pcr_y_test = y_train.iloc[pcr_train_index], y_train.iloc[pcr_test_index]
    PCs_train = pca.fit_transform(pcr_X_train)
    PCs_test = pca.transform(pcr_X_test)
    PCR_MSPE_train = []
    for j in range(1, 16):
      LModel.fit(PCs_train[:,:j], pcr_y_train)
      PCR_MSPE_train.append(mean_squared_error(pcr_y_test, LModel.predict(PCs_test[:,:j])))

  PCR_NCopm.append(np.argmin(PCR_MSPE_train))
  PCs_train = pca.fit_transform(X_train_std)
  PCs_test = pca.transform(X_test_std)
  LModel.fit(PCs_train[:,:PCR_NCopm[i]], y_train)
  PCR_MSPE.append(mean_squared_error(y_test, LModel.predict(PCs_test[:,:PCR_NCopm[i]])))

  # PLS
  for pls_train_index, pls_test_index in CV10.split(X_train_std):
    pls_X_train, pls_X_test = X_train_std[pcr_train_index], X_train_std[pcr_test_index]
    pls_y_train, pls_y_test = y_train.iloc[pcr_train_index], y_train.iloc[pcr_test_index]
    PLS_MSPE_train = []
    for j in range(1, 16):
      plsr = PLSRegression(n_components=j, scale=True)
      plsr.fit(pls_X_train, pls_y_train)
      PLS_MSPE_train.append(mean_squared_error(pls_y_test, plsr.predict(pls_X_test)))

  PLS_NCopm.append(np.argmin(PLS_MSPE_train))
  plsr = PLSRegression(n_components=PLS_NCopm[i], scale=True)
  plsr.fit(X_train_std, y_train)
  PLS_MSPE.append(mean_squared_error(y_test, plsr.predict(X_test_std)))


print(Ridge_MSPE)
print(LASSO_MSPE)
print(LASSO_NFeat)
print(PCR_MSPE)
print(PCR_NCopm)
print(PLS_MSPE)
print(PLS_NCopm)

RMSPEs and side-bu-side boxplots:

In [ ]:
RMSPE_Ridge = Ridge_MSPE / min(Ridge_MSPE)
RMSPE_LASSO = LASSO_MSPE / min(LASSO_MSPE)
RMSPE_PCR = PCR_MSPE / min(PCR_MSPE)
RMSPE_PLS = PLS_MSPE / min(PLS_MSPE)

print('RMSPE average for Ridge: ', np.mean(RMSPE_Ridge))
print('RMSPE average for LASSO: ', np.mean(RMSPE_LASSO))
print('RMSPE average for PCR: ', np.mean(RMSPE_PCR))
print('RMSPE average for PLS: ', np.mean(RMSPE_PLS))

print('RMSPE SD for Ridge: ', np.std(RMSPE_Ridge))
print('RMSPE SD for LASSO: ', np.std(RMSPE_LASSO))
print('RMSPE SD for PCR: ', np.std(RMSPE_PCR))
print('RMSPE SD for PLS: ', np.std(RMSPE_PLS))

RMSPEs = [RMSPE_Ridge, RMSPE_LASSO, RMSPE_PCR, RMSPE_PLS]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.boxplot(RMSPEs)
ax.set_xticklabels(['Ridge','LASSO','PCR', 'PLS'])